In [1]:
import pickle
import os

import numpy as np
import pandas as pd

use_mnli = False
path = 'paper_results_mnli/' if use_mnli else 'paper_results/'

# get all the files pickle in the path directory
files = os.listdir(path)
files = [f for f in files if f.endswith('.csv')]
#files

In [2]:
# order list of files
files = sorted(files)

# remove file that contain val in the name
files = [f for f in files if 'val' not in f]
#files

In [3]:
# read test.model-agnostic.json 
from datasets import load_dataset, concatenate_datasets, DatasetDict

test = load_dataset("json", data_files=["test.model-agnostic.json"])

/data1/malto/csavelli/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Table Results

In [4]:
ground_truth = test['train']['label']

In [ ]:
results = []

for i, f in enumerate(files): 
    df = pd.read_csv(os.path.join(path, f))
    # take the first column of the dataframe
    col = df.columns[0]
    preds = df[col].values
    preds = np.array(preds)

    # for every element of pred, if it is >0.5, then it is 1, else 0
    preds = np.where(preds > 0.5, 'Hallucination', 'Not Hallucination')
    results.append(np.sum(preds == ground_truth))
    #name = f.split('.')[0]
    #(name, np.sum(preds == ground_truth))

    if len(results) == 5:
        results = np.array(results)
        results = results / len(ground_truth)
        results = results * 100
        results = np.round(results, 2)

        name = f.split('.')[0]
        print(name, results.mean(), "+-", results.std(ddof=1))
        results = []


In [ ]:
np.unique(preds, return_counts=True)

# Examples

In [6]:
ground_truth = test['train']['p(Hallucination)']

In [9]:
name_file = 'sequential_test0.csv'

In [31]:
vectors = []

for i in range(5):
    name_file = f'sequential_test{i}.csv'
    df = pd.read_csv(os.path.join(path, name_file))
    # take the first column of the dataframe
    col = df.columns[0]
    preds = df[col].values
    preds = np.array(preds)
    print(preds[0])
    vectors.append(preds)

vectors = np.array(vectors)
vectors = np.mean(vectors, axis=0)
vectors[0]

0.11857715
0.098158024
0.12029305
0.0801174
0.14421432


0.1122719888

In [32]:
# find the most similar index between preds and ground_truth
most_similar_idx = np.argmin(np.abs(preds - ground_truth))
less_similar_idx = np.argmax(np.abs(preds - ground_truth))

most_similar_idx, less_similar_idx

(1104, 47)

In [41]:
# get the most similar and less similar examples in the dataset 
most_similar = test['train'].select([most_similar_idx])
print(most_similar['p(Hallucination)'][0], preds[most_similar_idx])
print(most_similar['src'])
print(most_similar['hyp'])
print(most_similar['tgt'])
print(most_similar['task'])

0.4 0.40015212
['Король Харальд Гормссон, более известный как Харальд Синезубый, ввёл в Дании христианство.']
['King Harald Hormsson, better known as Harald Sinezubii, introduced Christianity to Denmark.']
['King Harald Gormsson, better known as “Harald Bluetooth", introduced Christianity to Denmark.']
['MT']


In [40]:
less_similar = test['train'].select([less_similar_idx])
print(less_similar['p(Hallucination)'][0], preds[less_similar_idx])
print(less_similar['src'])
print(less_similar['hyp'])
print(less_similar['tgt'])
print(less_similar['task'])

0.0 0.90984696
["Why'd you got to go and do that?"]
['Why did you have to go do that?']
['Why would you say that?']
['PG']
